### REF: 

* https://medium.com/@shengyuchen/how-tfidf-scoring-in-content-based-recommender-works-5791e36ee8da
* https://towardsdatascience.com/content-based-recommender-systems-28a1dbd858f5

In [1]:
!pip install nltk==3.4
!pip install pandas==1.1.5
!pip install numpy==1.19.5
!pip install scikit-learn==0.24.2

In [2]:
import pandas as pd
import numpy as np
# Informado via documentação sobre o \t
df = pd.read_csv('../../data/base_info_produtos.csv', sep='\t')

df_model = df.copy()

In [3]:
# Exibe as dez primeiras linhas do dataset
df.head(10)

,nome,tipo,marca,categoria,cor,modelo
0,Samsung UN40C6900 LED Plana 40 Polegadas,TV,Samsung,Eletrônicos,None,None
1,Sapateira Limeira Alta 30 Pares Tabaco - Polit...,Armario,Politorno,Casa e Decoração,None,None
2,Faqueiro Tramontina Inox Laguna 66906760 20,Faqueiro,Tramontina,Casa e Decoração,Inox,None
3,Cartucho de tinta HP 72 C9371A 130 Mls Ciano,Cartucho,HP,None,None,None
4,Bolsa Pure Evo Medium Grip branco e preto - Puma,Bolsa,Puma,Moda e Acessórios,None,None
5,Docking Station 8W Branca 110/220V - Maxprint,Docking Station,Maxprint,Eletrônicos,None,None
6,Bandeirante Ecojipe,Mini Veículo,Bandeirante,Brinquedos,None,None
7,Câmera Digital Samsung ST95 16.2 Megapixels,Câmera Digital,Samsung,Eletrônicos,Preto,42lw4500
8,Ducha Manual LorenForma C/ Desviador - Lorenzetti,Chuveiro,Lorenzetti,None,None,None
9,Espagueteira de Alumínio Cozivapor Antiaderent...,Espagueteira,MTA,None,None,None


In [4]:
# Exibe quantidade de dados não nulos para cada variável
df_model.replace('None', np.nan).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26241 entries, 0 to 26240
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nome       26241 non-null  object
 1   tipo       25980 non-null  object
 2   marca      26241 non-null  object
 3   categoria  22732 non-null  object
 4   cor        4019 non-null   object
 5   modelo     5876 non-null   object
dtypes: object(6)
memory usage: 1.2+ MB


In [5]:
# Baixando e carregando a lista de stopwords pontuacao e pacotes de NLP
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')
from nltk.corpus import stopwords
from string import punctuation
from typing import List
import re

# Declarando o stemmer para pt
stemmer = nltk.stem.RSLPStemmer()

# Declarando as stop words customizadas
custom_stop_words = [
    'c/',
    's/',
    'p/'
]

stop_words = list(set(stopwords.words('portuguese') + list(punctuation) + custom_stop_words))

def custom_tokenizer(text_in: str,
                     min_leng: int = 1,
                     stop_words: List[str] = stop_words) -> str:
    
    text_in = " ".join(re.sub(r'[^\w]', ' ', text_in.lower()).split())
    bag: List[str] = nltk.word_tokenize(text_in)
    bag = [re.sub(r'[^\w]', ' ', word) for word in bag]
    bag = [stemmer.stem(word) for word in bag if word not in stop_words if len(word) > min_leng]
    return list(set(bag))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [6]:
# SUbstitui None por vazio
df_model = df_model.replace('None', "").copy()

# Seleciona as colunas que farao a composicao textual
prop_columns = ['nome','tipo','marca','categoria','cor','modelo']

# JUnta as colunas textuais
df_model['composition'] = df_model[prop_columns].apply(lambda x: " ".join(x), axis=1)

# Aplica tokenizacao, stemming e outros processamentos
df_model['pre_composition'] = df_model['composition'].apply(lambda x: custom_tokenizer(x))

In [7]:
df_model.head(10)

,nome,tipo,marca,categoria,cor,modelo,composition,pre_composition
0,Samsung UN40C6900 LED Plana 40 Polegadas,TV,Samsung,Eletrônicos,,,Samsung UN40C6900 LED Plana 40 Polegadas TV Sa...,"[poleg, un40c6900, tv, samsung, 40, plan, led,..."
1,Sapateira Limeira Alta 30 Pares Tabaco - Polit...,Armario,Politorno,Casa e Decoração,,,Sapateira Limeira Alta 30 Pares Tabaco - Polit...,"[politorn, tabac, alt, cas, decor, par, armari..."
2,Faqueiro Tramontina Inox Laguna 66906760 20,Faqueiro,Tramontina,Casa e Decoração,Inox,,Faqueiro Tramontina Inox Laguna 66906760 20 Fa...,"[cas, decor, inox, tramontin, faqu, lagun, 20,..."
3,Cartucho de tinta HP 72 C9371A 130 Mls Ciano,Cartucho,HP,,,,Cartucho de tinta HP 72 C9371A 130 Mls Ciano C...,"[cian, c9371, tint, 72, 130, ml, cartuch, hp]"
4,Bolsa Pure Evo Medium Grip branco e preto - Puma,Bolsa,Puma,Moda e Acessórios,,,Bolsa Pure Evo Medium Grip branco e preto - Pu...,"[mod, evo, pret, pur, acessóri, pum, medium, g..."
5,Docking Station 8W Branca 110/220V - Maxprint,Docking Station,Maxprint,Eletrônicos,,,Docking Station 8W Branca 110/220V - Maxprint ...,"[8w, docking, 110, station, 220v, maxprint, el..."
6,Bandeirante Ecojipe,Mini Veículo,Bandeirante,Brinquedos,,,Bandeirante Ecojipe Mini Veículo Bandeirante B...,"[ecojip, min, bandeir, veícul, brinqued]"
7,Câmera Digital Samsung ST95 16.2 Megapixels,Câmera Digital,Samsung,Eletrônicos,Preto,42lw4500,Câmera Digital Samsung ST95 16.2 Megapixels Câ...,"[digit, megapixel, pret, samsung, 42lw4500, câ..."
8,Ducha Manual LorenForma C/ Desviador - Lorenzetti,Chuveiro,Lorenzetti,,,,Ducha Manual LorenForma C/ Desviador - Lorenze...,"[man, lorenform, desvi, duch, chuv, lorenzett]"
9,Espagueteira de Alumínio Cozivapor Antiaderent...,Espagueteira,MTA,,,,Espagueteira de Alumínio Cozivapor Antiaderent...,"[temper, tamp, alumíni, espaguet, vidr, mta, c..."


In [8]:
# Importa e configura para o tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

# Padrao da norma em l2 -> Ridge
vectorizer = TfidfVectorizer(preprocessor=lambda x: x, 
                             tokenizer=lambda x: x,
                             ngram_range=(1,3))

In [9]:
# Declara a string de entrada
input_str = custom_tokenizer('Câmera Digital Samsung ST95 16.2 Megapixels')

In [10]:
# Vetoriza as strings, nota-se que a string de entrada sera a posicao 0
tfidf_new = vectorizer.fit_transform([input_str]+list(df_model['pre_composition']))

In [11]:
# Multiplica os coeficientes dos produtos para o vetor da string de entrada
asa = (tfidf_new * tfidf_new[0].T).A

In [12]:
# Quantidade de recomendacoes
n_rec = 10

In [13]:
# Elenca os coeficientes mais proximos da string de entrada
indices = np.argpartition(asa.T[0], -(n_rec+1))[-(n_rec+1):]

# COntrucao da lista que possui os indices dos maiores coeficientes excluindo a entrada
indices = [item-1 for item in indices if item !=0]

# Exibe as recomendacoes
df_model['nome'].iloc[indices]

17891    Câmera Digital Nikon Coolpix S6100 16.0 Megapi...
11771         Câmera Digital Sony DSC-TX10 16.2 Megapixels
3650            Câmera Digital Nikon D5100 16.2 Megapixels
12578          Câmera Digital Samsung NV10 10.1 Megapixels
18949          Câmera Digital Samsung ES28 12.2 Megapixels
20595         Câmera Digital Samsung PL120 14.2 Megapixels
7              Câmera Digital Samsung ST95 16.2 Megapixels
16722                 Câmera Digital 16.2MP ST95 - Samsung
21942          Câmera Digital Samsung ST93 16.1 Megapixels
14889        Câmera Digital ST95 - 16.2MP - Rosa - Samsung
Name: nome, dtype: object

In [14]:
# Contruindo a funcao
from typing import Any

def recommendation_list(product: str, n_rec: int = 10) -> pd.DataFrame:
    token_list: List[str] = custom_tokenizer(product)
    tf_idf: List[List[Any]] = vectorizer.fit_transform(
        [token_list]+list(df_model['pre_composition'])
    )
    tf_idf = (tf_idf * tf_idf[0].T).A
    index_list = np.argpartition(tf_idf.T[0], -(n_rec+1))[-(n_rec+1):]
    index_item_list: List[int] = [item-1 for item in index_list if item !=0]
    coeficient_list: List[float] = tf_idf.T[0][index_list]
    return pd.DataFrame(
        zip(list(df_model['nome'].iloc[index_item_list]), coeficient_list),
        columns=['Product', 'Score']
    ).sort_values(by='Score', ascending=False)

In [15]:
recommendation_list('Ducha Manual LorenForma C/ Desviador - Lorenzetti', 10)

,Product,Score
2,Ducha Manual LorenForma C/ Desviador - Lorenzetti,0.739395
7,Ducha Manual LorenQuadra com Desviador - Loren...,0.237310
1,Ducha Manual LorenStorm C/ Desviador - Lorenzetti,0.230774
9,Ducha Manual Joy com Desviador - Lorenzetti,0.230567
3,Ducha Loren Java com Haste e Desviador Lorenzetti,0.188653
5,Ducha Lorenzetti Lorenstorm 1986 C16 Com Desvi...,0.162001
8,Ducha Eletrônica Lorenzetti Evolution Master C...,0.141939
6,Ducha Higiênica Monocomando LorenStilo - Loren...,0.140279
4,Ducha Digital Temperatura Programável c/ Desvi...,0.139349
0,Ducha Eletrônica Temperatura Programável c/ De...,0.138390
